# 2018-06-06 - Psychophysics using psychopy

On repasse sur un paradigme plus classique de psychophysique.

On récupère les initiales :

In [1]:
exp_info = '2018-06-06_0648_LP'
print (exp_info)

2018-06-06_0648_LP


On défini la génération de MotionClouds :

In [2]:
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = '/tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

On définit les paramètres et on teste :

In [3]:
np.lookfor('clip')

Search results for 'clip'
-------------------------
numpy.clip
    Clip (limit) the values in an array.
numpy.chararray.clip
    Return an array whose values are limited to ``[min, max]``.
numpy.ma.MaskedArray.clip
    Return an array whose values are limited to ``[min, max]``.
numpy.put
    Replaces specified elements of an array with given values.
numpy.take
    Take elements from an array along an axis.
numpy.choose
    Construct an array from an index array and a set of arrays to choose from.
numpy.kaiser
    Return the Kaiser window.
numpy.hamming
    Return the Hamming window.
numpy.hanning
    Return the Hanning window.
numpy.bartlett
    Return the Bartlett window.
numpy.blackman
    Return the Blackman window.
numpy.corrcoef
    Return Pearson product-moment correlation coefficients.
numpy.ma.choose
    Use an index array to construct a new array from a set of choices.
numpy.ravel_multi_index
    Converts a tuple of index arrays into an array of flat
numpy.ma.MaskedArray.put
 

In [ ]:
from psychopy import visual, core, event
import MotionClouds as MC

test_length = 150  # number of trials
#trial_length = 2.6  # length of trial, in second
fixation_length = .5  # length of fixation, in second
stim1_length = 0.25  # length of first stim, in second
stim2_length = 0.25  # length of the second stim, in second

fullscr = False

fullscr = True

# Psychopy window
win = visual.Window(fullscr=fullscr, color=[0,0,0], winType='pygame', monitor='testMonitor', units='height')
mouse = event.Mouse(visible=False)

fixation = visual.ShapeStim(vertices='cross', win=win, size=0.05, pos=[0, 0], fillColor='black', units='height')
fixation.autolog = False

MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
bitmap1 = visual.ImageStim(win, MC1, mask='gauss', size=0.8, units='height')
bitmap1.autolog = False
bitmap2 = visual.ImageStim(win, MC1, mask='gauss', size=0.8, units='height')
bitmap2.autolog = False
#bitmap3 = visual.ImageStim(win, '%s.png' %
#                           str(2 * (np.pi/3)), mask='circle')
#bitmap3.autolog = False


#msg = visual.TextStim(win, text='<- LEFT or RIGHT ->', color='black')
msg = visual.TextStim(win, text='?', color='black')

#shift_dict = {'right': 2,
#              'left':  3}


# Answer list
ans_list = []
N_B_thetas = 5
B_thetas = np.pi*np.logspace(-7, 0, N_B_thetas, base=2)
    
std_theta = np.pi/6
for trial in range(test_length):
    clock = core.Clock()
    fixation.draw()
    win.flip()
    
    # Shift to left (3) or to right (2) ?
    #shift = random.randint(2, 3)
    theta = np.clip(std_theta *  np.random.randn(), -np.pi/4, np.pi/4)

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    bitmap1.setImage(MC1)

    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2
    bitmap2.setImage(MC2)

    #MC3 = generate_random_cloud(2 * (np.pi/3), B_theta=B_theta)  # if shift = 3
    # MC to psychopy objects
    
    # Times the trial
    while clock.getTime() < fixation_length + stim1_length + stim2_length:
        if 0.0 <= clock.getTime() < fixation_length:  # fixation
            fixation.draw()
        if fixation_length + 0.01 <= clock.getTime() < fixation_length + stim1_length:
            bitmap1.draw()
        if fixation_length + stim1_length <= clock.getTime() < fixation_length + stim1_length + stim2_length:
            bitmap2.draw()
        win.flip()

    # display orientation choice
    msg.draw()
    win.flip()

    # answer using the keyboard
    while True:
        ans = event.waitKeys()

        if len(ans) > 0:
            if ans[0] in ['escape', 'q']:
                win.close()
                core.quit()
            elif ans[0] in ['left', 'right']:
                print('At trial ', trial, 'Angle=', '%.3f' % (theta*180/np.pi), 'answer is ', ans[0], '; bandwidth=', '%.3f' % (B_theta*180/np.pi))
                break

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    ans_list.append([trial, theta, ans[0], B_theta])

win.close()

At trial  0 Angle= -7.637 answer is  left ; bandwidth= 180.000
At trial  1 Angle= 9.324 answer is  right ; bandwidth= 15.910
At trial  2 Angle= 32.461 answer is  right ; bandwidth= 1.406
At trial  3 Angle= -11.671 answer is  left ; bandwidth= 53.514
At trial  4 Angle= 6.326 answer is  right ; bandwidth= 4.730
At trial  5 Angle= -9.070 answer is  left ; bandwidth= 4.730
At trial  6 Angle= -12.333 answer is  left ; bandwidth= 1.406
At trial  7 Angle= -25.155 answer is  left ; bandwidth= 1.406
At trial  8 Angle= 1.054 answer is  right ; bandwidth= 4.730
At trial  9 Angle= -12.386 answer is  left ; bandwidth= 180.000
At trial  10 Angle= 45.000 answer is  right ; bandwidth= 1.406
At trial  11 Angle= -14.263 answer is  left ; bandwidth= 15.910
At trial  12 Angle= -17.961 answer is  left ; bandwidth= 15.910
At trial  13 Angle= 11.114 answer is  left ; bandwidth= 53.514
At trial  14 Angle= 45.000 answer is  right ; bandwidth= 15.910
At trial  15 Angle= -23.064 answer is  left ; bandwidth= 180.

In [ ]:
import pickle
pickle.dump(ans_list, open('Psychophys_discrim_%s.p' % exp_info, 'wb'))